## Quick Guide to Performance and Parallel Programming

There are many options available to improve the performance of your Python codes.
The first thing to determine is what is limiting your computation. It could be CPU
speed (unlikely), memory limitations (out-of-core computing), or it could be data
transfer speed (waiting on data to arrive for processing). If your code is pure-Python,
then you can try running it with Pypy, which is is an alternative Python implementa-
tion that employs a just-in-time compiler. If your code does not experience a massive
speed-up with Pypy, then there is probably something external to the code that is
slowing it down (e.g., disk access or network access). If Pypy doesn’t make any
sense because you are using many compiled modules that Pypy does not support,
then there are many diagnostic tools available

Python has its own built-in profiler cProfile you can invoke from the command
line as in the following


```bash
python -m cProfile -o program.prof my_program.py
```

### Testing with the Fibonacci sequence

In [2]:
def fib(n):
    """Display the n first terms of Fibonacci sequence"""
    a, b = 0, 1
    i = 0
    fib_list = []
    fib_list.append(b)
    while i < n:
        a, b = b, a+b
        fib_list.append(b)
        i +=1
    print(fib_list[0])

In [9]:
%time fib(100000)

1
CPU times: user 254 ms, sys: 162 ms, total: 416 ms
Wall time: 414 ms


In [5]:
%prun fib(100000)

1
 

         100040 function calls in 0.445 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.347    0.347    0.357    0.357 <ipython-input-2-6bb8f8030fb8>:1(fib)
        1    0.088    0.088    0.445    0.445 <string>:1(<module>)
   100001    0.010    0.000    0.010    0.000 {method 'append' of 'list' objects}
        3    0.000    0.000    0.000    0.000 socket.py:333(send)
        1    0.000    0.000    0.445    0.445 {built-in method builtins.exec}
        3    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:366(write)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 iostream.py:300(_is_master_process)
        3    0.000    0.000    0.000    0.000 {method 'acquire' of '_thread.lock' objects}
        3    0.000    0.000    0.000    0.000 threading.py:1104(is_alive)
     

In [6]:
488*10e-3/0.445

10.96629213483146

## multiprocessing

Python has a multiprocessing module that is part of the standard library. This makes it easy
to spawn child worker processes that can break off and individually process small
parts of a big job. However, it is still your responsibility as the programmer to figure
out how to distribute the data for your algorithm.

In [10]:
def fib(n):
    """Display the n first terms of Fibonacci sequence"""
    a, b = 0, 1
    i = 0
    fib_list = []
    fib_list.append(b)
    while i < n:
        a, b = b, a+b
        fib_list.append(b)
        i +=1
    return(fib_list[0])

In [12]:
%time test = [fib(i) for i in range(100000,100020,1)]

CPU times: user 5.38 s, sys: 3.42 s, total: 8.8 s
Wall time: 8.8 s


In [15]:
from multiprocessing import Pool

In [19]:
pool = Pool(processes=8)

Process ForkPoolWorker-14:
Process ForkPoolWorker-12:
Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
Process ForkPoolWorker-16:
Process ForkPoolWorker-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/hvribeiro/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/hvribeiro/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/hvribeiro/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/hvribeiro/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/hvribeiro/anaconda/lib/python3.6/multiprocessing/process.py",

In [20]:
%time test2 = pool.map(fib, range(100000,100020,1))

CPU times: user 4.96 ms, sys: 2.62 ms, total: 7.59 ms
Wall time: 2.2 s


In [18]:
pool.terminate()

## dask


In [92]:
!pip install dask

    100% |████████████████████████████████| 36.9MB 700kB/s 
    100% |████████████████████████████████| 163kB 1.7MB/s 
  Running setup.py bdist_wheel for dash ... done
  Stored in directory: /home/hvribeiro/.cache/pip/wheels/06/af/07/6f112f3eb4d01b90e817d728799be80f36398567368aef27e3
  Running setup.py bdist_wheel for flask-compress ... done
  Stored in directory: /home/hvribeiro/.cache/pip/wheels/96/32/88/a1f6d9dd3c29570ab3a8acc0d556b3b20abcf3c623c868ce0a
  Running setup.py bdist_wheel for dash-renderer ... done
  Stored in directory: /home/hvribeiro/.cache/pip/wheels/4b/dc/d0/e1ae0332df5fb1f056153818831750032936535150fed856b5
  Running setup.py bdist_wheel for retrying ... done
  Stored in directory: /home/hvribeiro/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built dash flask-compress dash-renderer retrying
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
python-louvain 0.6 has requirement networkx<2.0.0, but you'll hav

In [21]:
%%time
test3 = []
for i in range(100000,100020,1):
    a = fib(i)
    test3.append(a)
test3

CPU times: user 5.02 s, sys: 2.8 s, total: 7.82 s
Wall time: 7.81 s


In [29]:
dask.__version__

'0.18.2'

In [22]:
import dask
from dask.distributed import Client, progress
client = Client()

In [32]:
test4 = []
for i in range(100000,100020,1):
    a = dask.delayed(fib)(i)
    test4.append(a)
res = dask.delayed()(test4)

In [33]:
res

Delayed('list-def7df13-c7e1-4d06-b03d-67363433e83b')

open http://localhost:8787/status to see the progress

In [28]:
%time res.compute()

CPU times: user 460 ms, sys: 41.5 ms, total: 501 ms
Wall time: 3.05 s


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [34]:
res = res.persist()  # start computation in the background

In [35]:
res.compute()

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
